In [14]:
import os
from dotenv import load_dotenv
import requests

load_dotenv()
API_KEY = os.environ.get("OPENROUTER_API_KEY")
API_URL = "https://openrouter.ai/api/v1/chat/completions"

os.environ["GRAPH_DATABASE_PROVIDER"] = "kuzu"

In [3]:
import numpy as np
print(np.__version__)

1.26.4


In [11]:
import dlt

@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [12]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [13]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-09 02:13:33.842617+00:00 and COMPLETED in 8.02 seconds with 4 steps.
Step extract COMPLETED in 0.73 seconds.

Load package 1752027214.8809052 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.07 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)

Load package 1752027214.8809052 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.19 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.18 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/Workshop-dlt/db.qdrant location to store data
Load package 1752027214.8809052 is LOADED and contains no failed jobs

Step run COMPLETED in 8.02 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.18 seconds
1 load package(s) were loaded to destination qdrant and into dataset z